In [1]:
# import libraries
import pandas as pd
import numpy as np

In [2]:
# specify name of csv file
filename = 'final_data.csv'

# open csv file
df = pd.read_csv(filename)

df.head()

,Unnamed: 0.1,Unnamed: 0,PC6,Date,District,ConnectionTimeHours,kWh,effective_charging_hrs,power,MaxOccupancy,...,2019.0_household,2019.0_income_1_to_40_percent,2019.0_income_41_to_80_percent,2019.0_income_81_to_100_percent,2019.0_income_average,2019.0_assets_1_to_40_percent,2019.0_assets_41_to_80_percent,2019.0_assets_81_to_100_percent,2019.0_assets_average,2019.0_Gemiddelde SES WOA totaalscore
0,0,0,1011AB,2022-01-03,Centrum,67.59,125.8,22.789855,33.12,288.0,...,6200.0,45.0,28.0,27.1,49.3,50.8,23.0,26.2,47.6,0.034
1,66,66,1011AC,2022-01-03,0,0.00,0.0,0.000000,0.00,0.0,...,6200.0,45.0,28.0,27.1,49.3,50.8,23.0,26.2,47.6,0.034
2,132,18084,1011AD,2022-01-03,0,0.00,0.0,0.000000,0.00,0.0,...,6200.0,45.0,28.0,27.1,49.3,50.8,23.0,26.2,47.6,0.034
3,198,18150,1011AE,2022-01-03,0,0.00,0.0,0.000000,0.00,0.0,...,6200.0,45.0,28.0,27.1,49.3,50.8,23.0,26.2,47.6,0.034
4,264,132,1011AG,2022-01-03,0,0.00,0.0,0.000000,0.00,0.0,...,6200.0,45.0,28.0,27.1,49.3,50.8,23.0,26.2,47.6,0.034


In [3]:
# number of unique values in first column
print(df[df.columns[0]].nunique())

# number of unique values in second column
print(df[df.columns[1]].nunique())

# one column of index can be dropped, since it is not the correct index

1219812
1219812


In [4]:
# create a new column called "WeekIndex" which first lists all unique values of the date column, then assigns a number to each unique value starting from 0 for the lowest value
df['WeekIndex'] = df['Date'].astype('category').cat.codes

In [5]:
# print first 5 rows
df.rename(columns={df.columns[0]: 'index_to_drop', df.columns[1]: 'RowIndex'},inplace=True)

# remove "index" column, since this is not the correct index
df.drop(columns=["index_to_drop"], inplace=True)

In [6]:
# unit test that each row has a unique index
assert df['RowIndex'].nunique() == len(df.index)

# convert 'RowIndex' column to integer
df['RowIndex'] = df['RowIndex'].astype(int)

# set the 'RowIndex' column to index
df = df.set_index("RowIndex")

In [7]:
# cut district column and insert as last column
df['District'] = df['District'].str[0:2]

# count unique categories in District column -> 11
print(df["District"].nunique())

# encode categorical variable "District" with binary encoding
# documentation: https://contrib.scikit-learn.org/category_encoders/binary.html
from category_encoders.binary import BinaryEncoder
encoder = BinaryEncoder(cols=['District'])
encoder.fit(df)
df = encoder.transform(df)

9


In [8]:
# count unique categories in ZipCode "PC6" column
print(df["PC6"].nunique())

# retrieve PC4 from PC6, that is the first 4 digits of the 6 digit zip code
df["PC4"] = df["PC6"].astype(str).str[:4]
print(df["PC4"].nunique())

# encode categorical variable "PC4" with binary encoding
encoder = BinaryEncoder(cols=['PC4'])
encoder.fit(df)
df = encoder.transform(df)

18482
81


In [9]:
df.head()

,PC6,Date,District_0,District_1,District_2,District_3,ConnectionTimeHours,kWh,effective_charging_hrs,power,...,2019.0_assets_average,2019.0_Gemiddelde SES WOA totaalscore,WeekIndex,PC4_0,PC4_1,PC4_2,PC4_3,PC4_4,PC4_5,PC4_6
RowIndex,,,,,,,,,,,,,,,,,,,,,
0,1011AB,2022-01-03,0,0,0,1,67.59,125.8,22.789855,33.12,...,47.6,0.034,0,0,0,0,0,0,0,1
66,1011AC,2022-01-03,0,0,1,0,0.00,0.0,0.000000,0.00,...,47.6,0.034,0,0,0,0,0,0,0,1
18084,1011AD,2022-01-03,0,0,1,0,0.00,0.0,0.000000,0.00,...,47.6,0.034,0,0,0,0,0,0,0,1
18150,1011AE,2022-01-03,0,0,1,0,0.00,0.0,0.000000,0.00,...,47.6,0.034,0,0,0,0,0,0,0,1
132,1011AG,2022-01-03,0,0,1,0,0.00,0.0,0.000000,0.00,...,47.6,0.034,0,0,0,0,0,0,0,1


In [10]:
# all columns (including provider columns) that are not in the above lists are unknown variables
providers = ["Total", "Vattenfall", "Allego", "Equans", "EvBox", "Nuon", "WDS", "Pitpoint", "Ecotap", "Engie"]

# create a new column called "number of chargers" which is the sum of the values of the columns which names are stored in the providers list
df['number_of_chargers'] = df[providers].sum(axis=1)

providers.append("number_of_chargers")

# reverse list
providers = providers[::-1]

In [11]:
# split columns into several groups: identifiers, target variables, know_variables, unknown_variables, purely_descriptive_variables

# reorder columns in the following order:
identifiers = ["PC6", "Date"]
know_variables = ["WeekIndex", "MaxPower"]
target_variables = ["kWh", "Blocked_kWh"]

purely_descriptive_variables = ["ChargeSocket_ID_count", "ConnectionTimeHours", "power", "effective_charging_hrs", "MaxOccupancy", "SpareCap_Effective", "SpareCap_Occup_kWh", "SpareCap_Hrs", "Effective%", "Occupancy_kwh%"]


unknown_variables = [col for col in df.columns if col not in identifiers + target_variables + know_variables + providers+ purely_descriptive_variables]
unknown_variables = providers + unknown_variables

print(len(identifiers + target_variables + know_variables + purely_descriptive_variables + unknown_variables))

116


In [12]:
# sort df columns in the following order: identifiers, target variables, know_variables, unknown_variables, purely_descriptive_variables
df = df[identifiers + know_variables + target_variables + unknown_variables + purely_descriptive_variables]

In [13]:
# inspect df without the purely_descriptive_variables
df[identifiers + know_variables + target_variables + unknown_variables].head()

,PC6,Date,WeekIndex,MaxPower,kWh,Blocked_kWh,number_of_chargers,Engie,Ecotap,Pitpoint,...,2019.0_assets_81_to_100_percent,2019.0_assets_average,2019.0_Gemiddelde SES WOA totaalscore,PC4_0,PC4_1,PC4_2,PC4_3,PC4_4,PC4_5,PC4_6
RowIndex,,,,,,,,,,,,,,,,,,,,,
0,1011AB,2022-01-03,0,1589.76,125.8,373.0968,6.0,0.0,0.0,0.0,...,26.2,47.6,0.034,0,0,0,0,0,0,1
66,1011AC,2022-01-03,0,0.00,0.0,0.0000,0.0,0.0,0.0,0.0,...,26.2,47.6,0.034,0,0,0,0,0,0,1
18084,1011AD,2022-01-03,0,0.00,0.0,0.0000,0.0,0.0,0.0,0.0,...,26.2,47.6,0.034,0,0,0,0,0,0,1
18150,1011AE,2022-01-03,0,0.00,0.0,0.0000,0.0,0.0,0.0,0.0,...,26.2,47.6,0.034,0,0,0,0,0,0,1
132,1011AG,2022-01-03,0,0.00,0.0,0.0000,0.0,0.0,0.0,0.0,...,26.2,47.6,0.034,0,0,0,0,0,0,1


In [14]:
# check missing values: show only columns where missing values >0 and their count per column
print(df.isna().sum()[df.isna().sum() > 0])

Series([], dtype: int64)


In [15]:
# sort df by date column
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values(by=['PC6', 'Date'])

In [16]:
# add week of year column, extracted from date column
df["WeekOfYear"] = pd.to_datetime(df['Date']).dt.isocalendar().week

# month of year
df["MonthOfYear"] = pd.to_datetime(df['Date']).dt.month

# sine and cosine week of year -> to introduce cyclical nature of months
# sine & cosine week of year
# cosine week of year
df["CosWeekOfYear"] = np.cos(2*np.pi*df["WeekOfYear"]/52)

# sine week of year
df["SinWeekOfYear"] = np.sin(2*np.pi*df["WeekOfYear"]/52)

# cosine month of year
df["CosMonthOfYear"] = np.cos(2*np.pi*df["MonthOfYear"]/12)

# sine month of year
df["SinMonthOfYear"] = np.sin(2*np.pi*df["MonthOfYear"]/12)

In [17]:
df.head()

,PC6,Date,WeekIndex,MaxPower,kWh,Blocked_kWh,number_of_chargers,Engie,Ecotap,Pitpoint,...,SpareCap_Occup_kWh,SpareCap_Hrs,Effective%,Occupancy_kwh%,WeekOfYear,MonthOfYear,CosWeekOfYear,SinWeekOfYear,CosMonthOfYear,SinMonthOfYear
RowIndex,,,,,,,,,,,,,,,,,,,,,
0,1011AB,2022-01-03,0,1589.76,125.80,373.0968,6.0,0.0,0.0,0.0,...,-648.8208,220.41,0.079131,1.408125,1,1,0.992709,0.120537,0.866025,0.5
1,1011AB,2022-01-10,1,3709.44,457.95,904.3416,14.0,0.0,0.0,0.0,...,-8951.3424,508.17,0.123455,3.413125,2,1,0.970942,0.239316,0.866025,0.5
2,1011AB,2022-01-17,2,3709.44,525.80,1298.3040,14.0,0.0,0.0,0.0,...,-14466.8160,436.80,0.141746,4.900000,3,1,0.935016,0.354605,0.866025,0.5
3,1011AB,2022-01-24,3,3709.44,707.90,1855.3824,14.0,0.0,0.0,0.0,...,-22265.9136,335.88,0.190837,7.002500,4,1,0.885456,0.464723,0.866025,0.5
4,1011AB,2022-01-31,4,3709.44,571.31,1433.9304,14.0,0.0,0.0,0.0,...,-16365.5856,412.23,0.154015,5.411875,5,1,0.822984,0.568065,0.866025,0.5


In [18]:
# count number of remaining rows
print(df.shape)

(1219812, 122)


In [19]:
# load cleaned data
# df = pd.read_csv(filename.split('.')[0] + '_cleaned.csv')

# create new column as concatenation of PC6 and WeekIndex columns
df['PC6_WeekIndex'] = df['PC6'].astype(str) + df['WeekIndex'].astype(str)

# move this column to the first position
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]

# count duplicates in PC6_WeekIndex column
df['PC6_WeekIndex'].duplicated().sum()

0

In [23]:
# keep only first 10000 rows for testing purposes
# df = df.head(10000)

# inspect data
df.head(100)

,PC6,Date,WeekIndex,MaxPower,kWh,Blocked_kWh,number_of_chargers,Engie,Ecotap,Pitpoint,...,SpareCap_Hrs,Effective%,Occupancy_kwh%,WeekOfYear,MonthOfYear,CosWeekOfYear,SinWeekOfYear,CosMonthOfYear,SinMonthOfYear,PC6_WeekIndex
RowIndex,,,,,,,,,,,,,,,,,,,,,
0,1011AB,2022-01-03,0,1589.76,125.80,373.0968,6.0,0.0,0.0,0.0,...,220.41,0.079131,1.408125,1,1,0.992709,0.120537,0.866025,0.500000,1011AB0
1,1011AB,2022-01-10,1,3709.44,457.95,904.3416,14.0,0.0,0.0,0.0,...,508.17,0.123455,3.413125,2,1,0.970942,0.239316,0.866025,0.500000,1011AB1
2,1011AB,2022-01-17,2,3709.44,525.80,1298.3040,14.0,0.0,0.0,0.0,...,436.80,0.141746,4.900000,3,1,0.935016,0.354605,0.866025,0.500000,1011AB2
3,1011AB,2022-01-24,3,3709.44,707.90,1855.3824,14.0,0.0,0.0,0.0,...,335.88,0.190837,7.002500,4,1,0.885456,0.464723,0.866025,0.500000,1011AB3
4,1011AB,2022-01-31,4,3709.44,571.31,1433.9304,14.0,0.0,0.0,0.0,...,412.23,0.154015,5.411875,5,1,0.822984,0.568065,0.866025,0.500000,1011AB4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1011AC,2022-07-25,29,0.00,0.00,0.0000,0.0,0.0,0.0,0.0,...,0.00,0.000000,0.000000,30,7,-0.885456,-0.464723,-0.866025,-0.500000,1011AC29
96,1011AC,2022-08-01,30,0.00,0.00,0.0000,0.0,0.0,0.0,0.0,...,0.00,0.000000,0.000000,31,8,-0.822984,-0.568065,-0.500000,-0.866025,1011AC30
97,1011AC,2022-08-08,31,0.00,0.00,0.0000,0.0,0.0,0.0,0.0,...,0.00,0.000000,0.000000,32,8,-0.748511,-0.663123,-0.500000,-0.866025,1011AC31


In [21]:
# save to new csv file named original name + _cleaned
df.to_csv(filename.split('.')[0] + '_cleaned.csv', index=False)